In [1]:
import pandas as pd
import numpy as np

#### 平台新上了一个API，是获取融资融券数据的，用notebook简单做一个介绍吧
#### API文档在这里：https://www.ricequant.com/api/research/chn#research-API-get_price_change_rate
#### get_securities_margin(id_or_symbols, start_date=None, end_date=None, fields=None) 
首先对传入参数做一个简单的介绍：
   * id_or_symbols 传入你要查找的股票或者股票列表
   * start_date 起始查找日期 (可不填，返回最近这一个月的数据)
   * end_date 介绍查找日期(可不填，返回最近这一个月的数据)
   * fields() 查找你对应需要的数据 
        * margin_balance	融资余额
        * buy_on_margin_value	融资买入额
        * margin_repayment	融资偿还额
        * short_balance	融券余额
        * short_balance_quantity	融券余量
        * short_sell_value	融券卖出额
        * short_sell_quantity	融券卖出量
        * short_repayment_quantity	融券偿还量
        * total_balance	融资融券余额

### 目标1：查找单一股票的两融数据(返回一个dataFrame) 
### 目标2：查找多只股票的两融数据(返回一个dataPanel)
### 目标3：找出融资余额连续增长3天的股票




In [2]:
get_securities_margin('000012.XSHE',start_date='2016-11-15',end_date='2016-12-01').head()


,margin_balance,buy_on_margin_value,short_sell_quantity,margin_repayment,short_balance_quantity,short_repayment_quantity,short_balance,total_balance
2016-11-15,1.033709e+09,156766289.0,50800.0,NaN,85900.0,NaN,1047121.0,1.034756e+09
2016-11-16,1.058266e+09,171217188.0,2500.0,NaN,37000.0,NaN,452510.0,1.058719e+09
2016-11-17,1.237856e+09,463501332.0,81400.0,NaN,114700.0,NaN,1542715.0,1.239399e+09
2016-11-18,1.240697e+09,565062429.0,21400.0,NaN,69800.0,NaN,972314.0,1.241670e+09
2016-11-21,1.473361e+09,883424729.0,89300.0,NaN,114300.0,NaN,1751076.0,1.475112e+09


In [16]:
stocks=index_components('上证50')
DP=get_securities_margin(stocks,start_date='2016-11-15',end_date='2016-12-01')
DP

<class 'pandas.core.panel.Panel'>
Dimensions: 8 (items) x 13 (major_axis) x 50 (minor_axis)
Items axis: margin_balance to total_balance
Major_axis axis: 2016-11-15 00:00:00 to 2016-12-01 00:00:00
Minor_axis axis: 600104.XSHG to 600340.XSHG

In [19]:
stocks[:5]

['600104.XSHG', '601169.XSHG', '600100.XSHG', '600958.XSHG', '600036.XSHG']

这里我们拿上证50成份股的两融数据，那么我们获得的就是一个DataPanel，然后我们还是可以通过拿特地给时间或者特地股票来得到一个dataFrame

In [20]:
DP.minor_xs('600104.XSHG').head()

,margin_balance,buy_on_margin_value,short_sell_quantity,margin_repayment,short_balance_quantity,short_repayment_quantity,short_balance,total_balance
2016-11-15,774658716.0,76463420.0,60100.0,53122540.0,454859.0,97700.0,NaN,NaN
2016-11-16,762240224.0,19212300.0,40200.0,31630792.0,447059.0,48000.0,NaN,NaN
2016-11-17,747158376.0,18169994.0,18200.0,33251842.0,428359.0,36900.0,NaN,NaN
2016-11-18,736288545.0,19232405.0,86122.0,30102236.0,509181.0,5300.0,NaN,NaN
2016-11-21,770366512.0,82600675.0,49800.0,48522708.0,433459.0,125522.0,NaN,NaN


In [21]:
DP.major_xs('2016-12-01').head()

,margin_balance,buy_on_margin_value,short_sell_quantity,margin_repayment,short_balance_quantity,short_repayment_quantity,short_balance,total_balance
600104.XSHG,1.186395e+09,307135496.0,409388.0,302253997.0,900727.0,55900.0,NaN,NaN
601169.XSHG,1.096382e+09,33007399.0,1546986.0,29665076.0,4450945.0,82500.0,NaN,NaN
600100.XSHG,3.646836e+09,215190458.0,5700.0,193586327.0,237000.0,42300.0,NaN,NaN
600958.XSHG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600036.XSHG,2.239652e+09,118715108.0,911100.0,72650988.0,2630035.0,191039.0,NaN,NaN


现在正式进入正题：找出融资余额三连涨的股票：
   * 1.全部股票拿出来
   * 2.拿两融数据
   * 3.融资余额连涨3天的股票

In [26]:
stocks=list(all_instruments(type='CS')['order_book_id'])
threeup=[]
for stock in stocks:
    margin_balance=get_securities_margin(stock)['margin_balance']
    if margin_balance[-3]>margin_balance[-2] and margin_balance[-2]>margin_balance[-1]:
        threeup.append(stock)
threeup[:5]

['000671.XSHE', '600151.XSHG', '000333.XSHE', '002437.XSHE', '600535.XSHG']

#### 截至12月6日，有378只股票是融资余额三连涨的呢~剩下的各位可以去摸索一下啦！

In [31]:
balance=get_securities_margin('000001.XSHE')

In [32]:
balance.columns

Index(['margin_balance', 'buy_on_margin_value', 'short_sell_quantity',
       'margin_repayment', 'short_balance_quantity',
       'short_repayment_quantity', 'short_balance', 'total_balance'],
      dtype='object')

In [38]:
df=balance['margin_balance']
df.shape

(24,)

## 连续n天上涨 得到n天移动窗口，保证这n格窗口是上升趋势

In [57]:
n=2
df=np.array(df)
d2=[df[i:i+n] for i in range(len(df)-n)]
re=[((i[1:]-i[:-1])>0).prod() for i in d2]
for i in range(n):
  re.append(0)
#re=[re.append(0) for i in range(n)]
re

[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]

In [72]:
df[np.array(re,dtype=bool)]

array([  3.19462340e+09,   3.23873481e+09,   3.23139237e+09,
         3.18584928e+09,   3.00064359e+09])